In [1]:
pip install dateparser

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pdfminer.six

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langdetect

Note: you may need to restart the kernel to use updated packages.


# Filmijob Capstone Project: Interview Question Scoring

## FilmiJob CV Parser (Getting the keywords)

In [129]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import os 
from dateparser.search import search_dates #para identificar la fecha de nacimiento


import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
#INCLUDE HERE THE LINK TO THE CV OF THE CANDIDATE: Magda.pdf

In [6]:
def convert_pdf_to_txt(path, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(path, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close()
    return text

text = convert_pdf_to_txt('Magda.pdf', pages=[0,1,2]) #Aquí añadiremos la ruta del CV

print(text)

MAGDALENA BRZEGOWA 
Krakow, Poland | (+48) 533885950 | magdabrzegowa98@gmail.com | Linkedin.com/in/magdalenabrzegowa 

PROFESSIONAL SUMMARY 
A dynamic, perseverant and with a constant drive to learn professional with experience in Market Tracking and 
Forecasting.  In  alliance  with  strong  work  ethic,  high  interpersonal  and  leadership  skills  prepared  to  work  on 
challenging, as well as stimulating projects in technological sector.  

EDUCATION 
IE SCHOOL OF SCIENCE AND TECHNOLOGY                                             
Masters in Business Analytics and Big Data 
•  Concentration in Smart Industries 
•  Specialised in Machine Learning 

                                                 MADRID, SPAIN 
               SEPT. 2021 – JULY 2022 

IE BUSINESS SCHOOL                                                                                                                                                 
Bachelor of Business Administration, Major in Finance      
•  Persona

In [7]:
# IDENTIFICAR IDIOMA DEL CV

from langdetect import detect
idiomacv = detect(text) #Identifica el idioma del CV entre 55 idiomas https://pypi.org/project/langdetect/
print("IDIOMA DEL CV: ",idiomacv)

IDIOMA DEL CV:  en


In [8]:
# IDENTIFICACIÓN DE HABILIDADES. Este dato también se puede usar para sacar keywords de las job descriptions

import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

nltk.download('stopwords')

# Importar aquí las KEYWORDS
SKILLS_DB = ["teaching", "pharmaceutical", "erp", "fulfillment", "positioning", "tax", "service delivery", "investigate", "editorial", "account management", "business process", "valid drivers license", "electronics", "pr", "public relations", "javascript", "assembly", "digital marketing", "linux", "facebook", "spreadsheets", "recruit", "proposal", "sharepoint", "data entry", "hotel", "ordering", "branding", "life cycle", "real estate", "relationship management", "researching", "process improvements", "chemistry", "saas", "cad", "sales experience", "mathematics", "customer-facing", "audio", "project management skills", "six sigma", "hospitality", "mechanical engineering", "auditing", "employee relations", "android", "security clearance", "licensing", "adobe", "fundraising", "repairs", "iso", "market research", "warehouse", "business strategy", "pmp", "data management", "quality control", "reconciliation", "css", "conversion", "business analysis", "financial analysis", "ecommerce", "business intelligence", "c++", "client service", "publishing", "supervising", "complex projects", "key performance indicators", "scrum", "photoshop", "sports", "e-commerce", "journalism", "data collection", "higher education", "marketing programs", "financial management", "business plans", "user experience", "client relationships", "cloud", "analytical skills", "cisco", "internal stakeholders", "product marketing", "regulatory requirements", "itil", "information security", "aviation", "supply chain management", "python", "accounts payable", "industry experience", "autocad", "purchase orders", "acquisitions", "tv", "instrumentation", "strategic direction", "law enforcement", "call center", "experiments", "technical skills", "human resource", "business cases", "build relationships", "invoicing", "support services", "marketing strategy", "operating systems", "biology", "start-up", "electrical engineering", "workflows", "routing", "non-profit", "marketing plans", "due diligence", "business management", "iphone", "algorithms", "architectures", "reconcile", "dynamic environment", "external partners", "asset management", "microsoft word", "emea", "intranet", "sops", "sas", "digital media", "prospecting", "financial reporting", "project delivery", "seo", "operational excellence", "standard operating procedures", "c#", "technical knowledge", "on-call", "talent management", "stakeholder management", "tablets", "cms", "analyze data", "financial statements", "microsoft office suite", "fitness", "case management", "value proposition", "industry trends", "rfp", "broadcast", "portfolio management", "fabrication", "ux", "financial performance", "customer requirements", "psychology", "marketing materials", "resource management", "physics", "mortgage", "development activities", "end user", "business planning", "root cause", "analysis", "leadership development", "relationship building", "sdlc", "on-boarding", "quality standards", "regulatory compliance", "aws", "kpi", "status reports", "product line", "drafting", "jira", "phone calls", "product knowledge", "business stakeholders", "technical issues", "admissions", "supervisory experience", "usability", "pharmacy", "commissioning", "project plan", "ms excel", "fda", "test plans", "variances", "ui", "financing", "travel arrangements", "internal customers", "medical device", "counsel", "inventory management", "performance metrics", "lighting", "outsourcing", "indesign", "performance improvement", "management consulting", "graphic design", "transport", "information management", ".net", "startup", "matrix", "front-end", "project planning", "business systems", "accounts receivable", "public health", "hris", "german", "instructional design", "in-store", "data center", "matlab", "employee engagement", "cost effective", "sales management", "api", "adobe creative suite", "twitter", "program development", "event planning", "cash flow", "strategic plans", "vendor management", "trade shows", "hotels", "segmentation", "contract management", "gis", "talent acquisition", "photography", "internal communications", "client services", "ibm", "financial reports", "product quality", "beverage", "strong analytical skills", "underwriting", "cpr", "mining", "sales goals", "chemicals", "deposits", "scripting", "migration", "software engineering", "mis", "therapeutic", "general ledger", "tableau", "ms project", "standardization", "retention", "spelling", "media relations", "os", "daily operations", "immigration", "product design", "etl", "field sales", "driving record", "peoplesoft", "benchmark", "quality management", "apis", "test cases", "internal controls", "telecom", "business issues", "research projects", "data quality", "strategic initiatives", "office software", "cfa", "co-op", "big data", "journal entries", "vmware", "help desk", "statistical analysis", "datasets", "alliances", "solidworks", "prototype", "lan", "sci", "budget management", "unix", "rfps", "flex", "gaap", "experimental", "cpg", "information system", "customer facing", "process development", "web services", "international", "travel", "revenue growth", "software development life cycle", "operations management", "computer applications", "risk assessments", "sales operations", "raw materials", "internal audit", "physical security", "sql server", "affiliate", "computer software", "manage projects", "business continuity", "litigation", "it infrastructure", "cost reduction", "small business", "annual budget", "ios", "html5", "real-time", "consulting experience", "circuits", "machine learning", "risk assessment", "dns", "cross-functional team", "public policy", "analyzing data", "consulting services", "cinema", "arts", "recursos humanos", "excel", "accounting", "acquisition", "administrative support", "advertising", "agile", "analytical", "analytics", "architecture", "audit", "automation", "banking", "bi", "billing", "brand", "budget", "budgeting", "business administration", "business development", "business requirements", "c (programming language)", "certification", "change management", "coaching", "coding", "compliance", "computer science", "construction", "consulting", "consumers", "content", "continuous improvement", "contracts", "controls", "correspondence", "counseling", "crm", "customer experience", "customer service", "d (programming language)", "data analysis", "database", "design", "distribution", "documentation", "documenting", "drawings", "economics", "editing", "electrical", "engagement", "engineering", "expenses", "filing", "finance", "financial services", "forecasting", "forecasts", "frameworks", "governance", "hardware", "health", "healthcare", "hing", "hospital", "html", "i-deas", "information systems", "information technology", "installation", "internship", "inventory", "investigation", "investigations", "invoices", "java", "kpis", "lean", "legal", "legislation", "lifecycle", "logistics", "man resources", "management experience", "marketing", "merchandising", "metrics", "microsoft office", "mobile", "modeling", "ms office", "negotiation", "networking", "nursing", "operations", "oracle", "outreach", "partnership", "partnerships", "payments", "payroll", "performance management", "policies", "presentation", "presentations", "procurement", "product development", "product management", "program management", "programming", "project management", "protocols", "publications", "purchasing", "qa", "quality assurance", "r (programming language)", "recruiting", "recruitment", "regulations", "regulatory", "reporting", "research", "retail", "risk management", "safety", "sales", "salesforce", "sap", "scheduling", "social media", "software development", "sourcing", "specifications", "sql", "staffing", "statistics", "strategic planning", "strategy", "supply chain", "technical", "technical support", "testing", "training", "transactions", "transportation", "troubleshooting", "vendors", "video", "windows", "writing", "accomplished", "achieved", "acquired", "acted", "adapted", "addressed", "adjusted", "administered", "advanced", "advertised", "advised", "advocated", "aided", "aligned", "allocated", "amplified", "amplify", "analyzed", "answered", "applied", "appointed", "appraised", "approved", "arbitrated", "architected", "arranged", "articulated", "assembled", "assessed", "assigned", "assisted", "attained", "audited", "augmented", "authored", "authorized", "automated", "awarded", "balanced", "began", "blocked", "boosted", "brainstormed", "briefed", "budgeted", "built", "calculated", "campaigned", "capitalized", "captured", "cared", "cataloged", "categorized", "centralized", "chaired", "championed", "charted", "checked", "clarified", "classified", "co-authored", "coached", "coded", "collaborated", "collected", "combined", "communicated", "compared", "compiled", "completed", "composed", "computed", "conceptualized", "condensed", "conducted", "conferred", "conserved", "considered", "consolidated", "constructed", "consulted", "contacted", "contracted", "contributed", "controlled", "convert", "converted", "conveyed", "convinced", "cooperated", "coordinated", "corrected", "corresponded", "counseled", "crafted", "created", "creative", "critiqued", "cultivated", "curated", "customized", "debated", "debugged", "decided", "deciphered", "decreased", "deducted", "defined", "delegated", "delivered", "demonstrated", "deployed", "derived", "described", "designed", "detected", "determined", "developed", "devised", "diagnosed", "diagramed", "differentiated", "directed", "discerned", "discovered", "discussed", "dispatched", "dispensed", "displayed", "distributed", "documented", "drafted", "drew", "drove", "earned", "edited", "educated", "elicited", "eliminated", "emphasized", "empowered", "enabled", "enacted", "encouraged", "endeavored", "endorsed", "enforced", "engineered", "enhanced", "enlisted", "enlivened", "ensured", "entertained", "equalized", "established", "estimated", "evaluated", "examined", "exceeded", "executed", "expanded", "expedited", "experimented", "explained", "explored"\
           , "extracted", "fabricated", "facilitated", "familiarize", "fashioned", "fielded", "filed", "finalized", "fixed", "focused", "forecasted", "forged", "formalized", "formed", "formulated", "fortified", "fostered", "founded", "furnished", "furthered", "gained", "gathered", "generated", "goals", "grew", "guided", "halted", "handled", "headed", "helped", "helping", "hired", "hosted", "identified", "ignited", "illustrated", "imagined", "implemented", "improved", "incorporated", "increased", "individualized", "influenced", "informed", "initiated", "inspected", "inspired", "installed", "instilled", "instituted", "instructed", "insured", "integrated", "intensified", "interacted", "interpreted", "intervened", "interviewed", "introduced", "invented", "inventoried", "investigated", "involved", "itemized", "joined", "judged", "launched", "lectured", "led", "lessened", "leveraged", "lifted", "listened", "lobbied", "located", "logged", "lowered", "maintained", "managed", "mapped", "marketed", "maximized", "measured", "mediated", "mentored", "merged", "minimized", "mobilized", "modeled", "moderated", "modified", "monitored", "motivated", "navigated", "negotiated", "networked", "observed", "obtained", "operated", "optimized", "orchestrated", "ordered", "organized", "originated", "outlined", "outpaced", "outperformed", "overhauled", "oversaw", "participated", "partnered", "patched", "performed", "persuaded", "photographed", "piloted", "pioneered", "planned", "predicted", "prepared", "presented", "presided", "printed", "prioritized", "probed", "processed", "produced", "programmed", "projected", "promoted", "proofread", "proposed", "proved", "provided", "publicized", "published", "purchased", "qualified", "quantified", "raised", "reached", "rebuilt", "recognized", "recommended", "reconciled", "recorded", "recruited", "rectified", "redesigned", "reduced", "referred", "refined", "refocused", "registered", "regulated", "rehabilitated", "reinforced", "remodeled", "renovated", "reorganized", "repaired", "replaced", "reported", "researched", "reserved", "resolved", "responded", "restored", "restructured", "retrieved", "revamped", "reviewed", "revised", "revitalized", "rewrote", "routed", "saved", "scheduled", "screened", "scrutinized", "searched", "secured", "set", "shaped", "sharpened", "shattered", "showcased", "simplified", "simulated", "slashed", "solicited", "solved", "sparked", "spearheaded", "specialized", "specified", "spoke", "standardized", "steered", "stimulated", "storyboarded", "strategized", "streamlined", "strengthened", "studied", "submitted", "succeeded", "suggested", "summarized", "supervised", "supplied", "supported", "surpassed", "surveyed", "sustained", "synthesized", "systematized", "targeted", "taught", "terminated", "tested", "tracked", "trained", "transformed", "translated", "transmitted", "troubleshoot", "tutored", "unified", "united", "up", "updated", "upgraded", "utilized", "validated", "verified", "visualized", "volunteered", "won", "wrote", "yielded", "accountable", "accurate", "active", "adept", "alert", "amiable", "amicable", "ardent", "articulate", "assertive", "astute", "attentive", "bold", "businesslike", "calculating", "calm", "charming", "cheerful", "clear", "coherent", "cohesive", "collaborative", "committed", "competent", "complex", "concerted", "confident", "conscientious", "consistent", "constructive", "cooperative", "cost-effective", "courteous", "cutting-edge", "decisive", "dedicated", "dependable", "detail-oriented", "detailed", "devoted", "diligent", "diplomatic", "discerning", "disciplined", "driven", "earnest", "economical", "elegant", "eloquent", "energetic", "enterprising", "enthusiastic", "exact", "exhaustive", "expert", "fast-learning", "fastidious", "first-class", "fluent", "genuine", "groundbreaking", "harmonious", "honest", "ideal", "imaginative", "impartial", "industrious", "influential", "ingenious", "innovative", "insightful", "inspiring", "instrumental", "intelligent", "inventive", "investigative", "keen", "logical", "loyal", "methodical", "meticulous", "novel", "objective", "orderly", "painstaking", "passionate", "perceptive", "persistent", "personable", "persuasive", "pleasant", "positive", "practical", "precise", "proactive", "productive", "professional", "proficient", "profitable", "progressive", "punctual", "purposeful", "quality", "relentless", "reliable", "resilient", "respectful", "revolutionary", "robust", "rousing", "shrewd", "significant", "sincere", "skilled", "skillful", "smooth", "sophisticated", "spirited", "steadfast", "step-by-step", "strategic", "strong-willed", "structured", "studious", "superior", "supportive", "sustainable", "systematic", "tactful", "team-minded", "tenacious", "thoughtful", "timely", "tireless", "unique", "unprecedented", "vigorous", "visionary", "vivid", "well-grounded", "wholehearted", "world-class", "zealous"]


def extract_skills(input_text):
    if idiomacv == 'en':
        stop_words = set(nltk.corpus.stopwords.words('english')) # Habría que adaptarlo con los diferentes idiomas
    elif idiomacv == 'es':
        stop_words = set(nltk.corpus.stopwords.words('spanish'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)

    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]

    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]

    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))

    # we create a set to keep the results in.
    found_skills = set()

    # we search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in SKILLS_DB:
            found_skills.add(token)

    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram in SKILLS_DB:
            found_skills.add(ngram)

    return found_skills

if __name__ == '__main__':
    skills = extract_skills(text)

print("SKILLS: ", skills)

SKILLS:  {'INTERNATIONAL', 'revised', 'fluent', 'Forecasting', 'lowered', 'Finance', 'FINANCE', 'TEACHING', 'Participated', 'prepared', 'SQL', 'warehouse', 'Prepared', 'Accounting', 'International', 'Conducted', 'market research', 'quality control', 'professional', 'Python', 'Analytics', 'adapted', 'strengthened', 'Provided', 'Taught', 'TECHNICAL', 'Created', 'Expanded', 'quality', 'RESEARCH', 'forecasting', 'PROFESSIONAL', 'analysis', 'research', 'Developed', 'contributed', 'Marketing', 'distribution', 'CONSULTING', 'Identified'}


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jdieg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jdieg\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\jdieg\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jdieg\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jdieg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
type(skills)

set

## EDA (Exploratory Data Analysis)

### Importing the roles dataset and checking the data

In [10]:
df=pd.read_csv("Dataset.csv")
df

,title,job description,questions
0,Collection Specialist,<h4>Job brief</h4><p>We are looking for a comp...,['A debtor has missed six months of payments. ...
1,Billing Analyst,<h4>Job brief</h4><p>We are looking for a Bill...,['How would you create a report on debts from ...
2,Tax Accountant,<h4>Job brief</h4><p>We are looking for a Tax ...,['What would you do if you received a B-Notice...
3,Accounts Receivable Manager,<h4>Job brief</h4><p>We are looking for an Acc...,['What would be the first thing you’d check if...
4,Cost Accountant,<h4>Job brief</h4><p>We are looking for a Cost...,"['If you could use only Excel, how would you o..."
...,...,...,...
278,Sales Advisor,"<h4>Job brief</h4><p><span style=""font-weight:...",['Imagine a customer asks you a question about...
279,Retail Buyer,"<h4>Job brief</h4><p><span style=""font-weight:...",['If I asked you to eliminate one line of our ...
280,Category Manager,"<h4>Job brief</h4><p><span style=""font-weight:...",['Here’s a spreadsheet with data. Can you spot...
281,Merchandiser,<h4>Job brief</h4><p>We are looking for an exp...,"['As a merchandiser, what do you think is the ..."


### Counting the number of keywords and checking for repeated keywords

In [11]:
#opened_file= open("keywords.txt")
from csv import reader
#read_file = reader(opened_file)
keywords_data = list(skills)

In [12]:
keywords_data

['INTERNATIONAL',
 'revised',
 'fluent',
 'Forecasting',
 'lowered',
 'Finance',
 'FINANCE',
 'TEACHING',
 'Participated',
 'prepared',
 'SQL',
 'warehouse',
 'Prepared',
 'Accounting',
 'International',
 'Conducted',
 'market research',
 'quality control',
 'professional',
 'Python',
 'Analytics',
 'adapted',
 'strengthened',
 'Provided',
 'Taught',
 'TECHNICAL',
 'Created',
 'Expanded',
 'quality',
 'RESEARCH',
 'forecasting',
 'PROFESSIONAL',
 'analysis',
 'research',
 'Developed',
 'contributed',
 'Marketing',
 'distribution',
 'CONSULTING',
 'Identified']

In [13]:
len(keywords_data)

40

### Creating a new column with the number of question per role

In [14]:
df["number_of_questions"]=df["questions"].str.split("[^a-e]', ?'[^a-e]").str.len()
df

,title,job description,questions,number_of_questions
0,Collection Specialist,<h4>Job brief</h4><p>We are looking for a comp...,['A debtor has missed six months of payments. ...,16
1,Billing Analyst,<h4>Job brief</h4><p>We are looking for a Bill...,['How would you create a report on debts from ...,14
2,Tax Accountant,<h4>Job brief</h4><p>We are looking for a Tax ...,['What would you do if you received a B-Notice...,14
3,Accounts Receivable Manager,<h4>Job brief</h4><p>We are looking for an Acc...,['What would be the first thing you’d check if...,17
4,Cost Accountant,<h4>Job brief</h4><p>We are looking for a Cost...,"['If you could use only Excel, how would you o...",15
...,...,...,...,...
278,Sales Advisor,"<h4>Job brief</h4><p><span style=""font-weight:...",['Imagine a customer asks you a question about...,16
279,Retail Buyer,"<h4>Job brief</h4><p><span style=""font-weight:...",['If I asked you to eliminate one line of our ...,19
280,Category Manager,"<h4>Job brief</h4><p><span style=""font-weight:...",['Here’s a spreadsheet with data. Can you spot...,15
281,Merchandiser,<h4>Job brief</h4><p>We are looking for an exp...,"['As a merchandiser, what do you think is the ...",15


In [15]:
df.describe()

,number_of_questions
count,283.000000
mean,15.342756
std,3.106703
min,7.000000
25%,13.000000
50%,15.000000
75%,17.000000
max,29.000000


On **average**, we have **15** questions per role. Also, the role with **less** questions has **7** and the role with the max questions has **29** questions.

### Cheking if there are any duplicated roles

In [16]:
all_jobs=df["title"].tolist()
all_jobs_set=set(all_jobs)
all_jobs_list=list(all_jobs_set)
len(all_jobs_list)

283

All the rows have a different role and no role is duplicated

### Checking if any of the columns has null values

In [17]:
df.isnull().any()

title                  False
job description        False
questions              False
number_of_questions    False
dtype: bool

No null values on the dataset

### Inspecting the total number of questions and questions repetitions

First, we are goint to split the questions in every row of the questions column into a list of lists:

In [18]:
df["questions"]=df["questions"].str.split("[^a-e]', ?'[^a-e]")  
df

,title,job description,questions,number_of_questions
0,Collection Specialist,<h4>Job brief</h4><p>We are looking for a comp...,[['A debtor has missed six months of payments....,16
1,Billing Analyst,<h4>Job brief</h4><p>We are looking for a Bill...,[['How would you create a report on debts from...,14
2,Tax Accountant,<h4>Job brief</h4><p>We are looking for a Tax ...,[['What would you do if you received a B-Notic...,14
3,Accounts Receivable Manager,<h4>Job brief</h4><p>We are looking for an Acc...,[['What would be the first thing you’d check i...,17
4,Cost Accountant,<h4>Job brief</h4><p>We are looking for a Cost...,"[['If you could use only Excel, how would you ...",15
...,...,...,...,...
278,Sales Advisor,"<h4>Job brief</h4><p><span style=""font-weight:...",[['Imagine a customer asks you a question abou...,16
279,Retail Buyer,"<h4>Job brief</h4><p><span style=""font-weight:...",[['If I asked you to eliminate one line of our...,19
280,Category Manager,"<h4>Job brief</h4><p><span style=""font-weight:...",[['Here’s a spreadsheet with data. Can you spo...,15
281,Merchandiser,<h4>Job brief</h4><p>We are looking for an exp...,"[['As a merchandiser, what do you think is the...",15


After having a list of questions for every role, we are going to create a list of all the questions in the dataset:

In [19]:
all_questions=[]
for questions in df["questions"]:
    for question in questions:
        all_questions.append(question)
all_questions

["['A debtor has missed six months of payments. When you call the individual, they bring up financial difficulties and get emotional. How do you handle this",
 'hat payback plan would you recommend for a long-term client whose payments are due to be collected in X months',
 'f you called a client to update them on their payment status and they were aggressive or rude to you, how would you handle it',
 'an you think of a few common excuses that debtors make? How would you respond to each of them',
 'hat collection methods do you know? Which technique do you think would be more effective for our company/clients',
 'ow do you keep historical data for each account',
 'ow often do you follow up with delinquent account holders and what’s your approach when you get in touch',
 'ention three things a Collection Specialist should avoid saying when speaking to a debtor',
 'ow do you keep track of all payments',
 'hat software do you use to increase your productivity',
 'hat do you find interesti

In [20]:
len(all_questions)

4342

There are **4342** total questions

In [21]:
set_allquestions=set(all_questions)
unique_questions=list(set_allquestions)
len(unique_questions)

4141

There are **4141** different questions

In [22]:
num_repeated_questions=len(all_questions)-len(unique_questions)
num_repeated_questions

201

There are **201** question repetitions

Now, we are going to create a frequency table of questions to double check total number and repetitions:

In [23]:
questions_freq={}
for questions in df["questions"]:
    for question in questions:
        if question in questions_freq.keys():
            questions_freq[question]+=1
        else:
            questions_freq[question]=1
            
questions_freq 

{"['A debtor has missed six months of payments. When you call the individual, they bring up financial difficulties and get emotional. How do you handle this": 1,
 'hat payback plan would you recommend for a long-term client whose payments are due to be collected in X months': 1,
 'f you called a client to update them on their payment status and they were aggressive or rude to you, how would you handle it': 1,
 'an you think of a few common excuses that debtors make? How would you respond to each of them': 1,
 'hat collection methods do you know? Which technique do you think would be more effective for our company/clients': 1,
 'ow do you keep historical data for each account': 1,
 'ow often do you follow up with delinquent account holders and what’s your approach when you get in touch': 1,
 'ention three things a Collection Specialist should avoid saying when speaking to a debtor': 1,
 'ow do you keep track of all payments': 1,
 'hat software do you use to increase your productivity': 

In this step we order the dictionary of questions in descending order to check the "most repeated" questions:

In [24]:
{k: v for k, v in sorted(questions_freq.items(), key=lambda item: item[1],reverse=True)}

{'ow would you rate your computer skills': 5,
 'ow do you ensure accuracy in routine tasks such as processing expenses and preparing reports': 4,
 'hat constitutes a good unit test and what a functional one': 4,
 'hat accounting software have you used': 3,
 'ow familiar are you with SFAS (Statement of Financial Accounting Standards)': 3,
 'ow fast can you type': 3,
 'escribe your experience with calendar management': 3,
 'hat tools do you use for linting, debugging and profiling': 3,
 'hat do we mean when we say that a certain Lambda expression forms a closure': 3,
 'hat tools & practices would you consider necessary for a Continuous Delivery solution': 3,
 'o arguments in Java get passed by reference or by value': 3,
 'ow are Runtime exceptions different from Checked exceptions': 3,
 'ervlets 3.0 introduced async support. Describe a use case for it': 3,
 'ow do you troubleshoot a crashing application': 3,
 'hat’s your current occupation/What are you currently working on': 3,
 'ake us 

In [25]:
count=0
repetitions=0
uniques=0
rep=[]
for key,value in questions_freq.items():
    if value>1:
        count+=1
        repetitions+=value
        rep.append(key)
    else:
        uniques+=1
print("There are",count,"questions with at least 2 appearances")
print("There are",uniques,"unique questions")
print("There are a total of ",(uniques+repetitions),"questions appearances")

There are 169 questions with at least 2 appearances
There are 3972 unique questions
There are a total of  4342 questions appearances


### Getting rid of the HTML code in the Job Description to make processing easier

In [91]:
df["job description"]=df["job description"].str.replace("<li>","")
df["job description"]=df["job description"].str.replace("</li>",", ")
df["job description"]=df["job description"].str.replace("</h4><p>",": ")
df["job description"]=df["job description"].str.replace("</h4><ul>",": ")
df["job description"]=df["job description"].str.replace("</p>","")
df["job description"]=df["job description"].str.replace("<h4>","")
df["job description"]=df["job description"].str.replace("</h4>",": ")
df["job description"]=df["job description"].str.replace("</ul>","")
df["job description"]=df["job description"].str.replace("\n","")
df["job description"]=df["job description"].str.replace("<p>"," ")
df["job description"]=df["job description"].str.replace('<span style="font-weight: 400;">',"")
df["job description"]=df["job description"].str.replace('</span><b>',"")
df["job description"]=df["job description"].str.replace('</span>',"")
df["job description"]=df["job description"].str.replace('</b>',"")
df["job description"]=df["job description"].str.replace(';,',",")
df["job description"]=df["job description"].str.replace('<p dir="ltr">',"")
df["job description"]=df["job description"].str.replace('<marketing-manager-job-description" rel="noopener" target="_blank">',"")
df["job description"]=df["job description"].str.replace('<marketing-specialist-job-description" rel="noopener" target="_blank">',"")
df["job description"]=df["job description"].str.replace('<marketing-officer-job-description" rel="noopener" target="_blank">',"")
df["job description"]=df["job description"].str.replace('<copywriter-job-description" rel="noopener" target="_blank">',"")
df["job description"]=df["job description"].str.replace('<data-analyst-job-description" rel="noopener" target="_blank">',"")
df["job description"]=df["job description"].str.replace('<accountant-job-description" >Accountants and <junior-accountant-job-description" >',"")
df["job description"]=df["job description"].str.replace('<financial-controller-job-description" >',"")
df["job description"]=df["job description"].str.replace('<financial-manager-job-description" >',"")
df["job description"]=df["job description"].str.replace('<accounting-manager-job-description" >',"")
df["job description"]=df["job description"].str.replace('<accounting-supervisor-job-description">',"")
df["job description"]=df["job description"].str.replace('<management-accountant-job-description" >',"")
df["job description"]=df["job description"].str.replace('<certified-public-accountant-cpa-job-description">',"")
df["job description"]=df["job description"].str.replace('<accounting-clerk-job-description">',"")
df["job description"]=df["job description"].str.replace('<medical-secretary-job-description">',"")
df["job description"]=df["job description"].str.replace('<span class="hardreadability"><span data-offset-key="65n7a-0-0"><span data-text="true">',"")


In [92]:
df["job description"][198]

'Job brief: We are looking for an experienced Procurement Manager to manage the company’s supply of products and services<span data-offset-key="65n7a-1-0"><span data-text="true">. Procurement Manager responsibilities include strategizing to\xa0find cost-effective deals and suppliers.\xa0<span class="hardreadability"><span data-offset-key="65n7a-2-0"><span data-text="true">The Procurement Manager’s duty\xa0is to discover the best\xa0ways to cut procurement expenses, so that the company can invest in growth and people<span data-offset-key="65n7a-3-0"><span data-text="true">.Responsibilities: <span data-offset-key="65n7a-0-0"><span data-text="true">Devise and use fruitful <sourcing-manager-job-description"><span data-offset-key="65n7a-1-0"><span data-text="true">sourcing strategies, <li class="public-DraftStyleDefault-unorderedListItem public-DraftStyleDefault-depth0 public-DraftStyleDefault-listLTR" data-block="true" data-editor="98src" data-offset-key="1qema-0-0"><div class="public-Draf

## Feature Engineering

### Normalization of keywords

#### Applying Lemmatization to the keywords list

In [35]:
# We still don't know if we are going to use it

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatized_keywords_list=[]
for keyword in keywords_data:
    lemmatized_keywords_list.append(lemmatizer.lemmatize(keyword))

In [36]:
lemmatized_keywords_list

['Provided',
 'market research',
 'Developed',
 'TECHNICAL',
 'Marketing',
 'Identified',
 'adapted',
 'Accounting',
 'lowered',
 'professional',
 'TEACHING',
 'forecasting',
 'PROFESSIONAL',
 'quality control',
 'warehouse',
 'International',
 'revised',
 'Created',
 'SQL',
 'Prepared',
 'Conducted',
 'Finance',
 'quality',
 'INTERNATIONAL',
 'prepared',
 'contributed',
 'fluent',
 'RESEARCH',
 'strengthened',
 'Expanded',
 'analysis',
 'Python',
 'Taught',
 'FINANCE',
 'Forecasting',
 'Analytics',
 'CONSULTING',
 'Participated',
 'research',
 'distribution']

#### Putting all keywords in lowercase

In [95]:
a = (map(lambda x: x.lower(), keywords_data))
lower_keyword_list= list(a)
print(lower_keyword_list)

['international', 'revised', 'fluent', 'forecasting', 'lowered', 'finance', 'finance', 'teaching', 'participated', 'prepared', 'sql', 'warehouse', 'prepared', 'accounting', 'international', 'conducted', 'market research', 'quality control', 'professional', 'python', 'analytics', 'adapted', 'strengthened', 'provided', 'taught', 'technical', 'created', 'expanded', 'quality', 'research', 'forecasting', 'professional', 'analysis', 'research', 'developed', 'contributed', 'marketing', 'distribution', 'consulting', 'identified']


### Grouping Keywords with job descriptions

In [101]:
df["job description"][0]

'Job brief: We are looking for a competent Collection Specialist to contact clients and collect outstanding payments. You will struck a balance between maintaining trustful relationships, and ensuring timely payments. Our Collection Specialist should exhibit professionalism and trustworthiness. You should have excellent communication and negotiation skills, as well as an ability to work independently.Responsibilities: Monitor accounts to identify outstanding debts, Investigate historical data for each debt or bill, Find and contact clients to ask about their overdue payments, Take actions to encourage timely debt payments, Process payments and refunds, Resolve billing and customer credit issues, Update account status records and collection efforts, Report on collection activity and accounts receivable status, Requirements: <collection-specialist-interview-questions">Proven experience as a Collection Specialist or similar role, Knowledge of billing procedures and collection techniques (

In [114]:
lower_job_description=[]
for role in df["job description"]:
    role=role.lower()
    lower_job_description.append(role)
print(lower_job_description[123])
df["job description"]=lower_job_description

job brief: we are looking for an experienced training and development manager to devise our organizational training strategy, oversee its implementation and assess its outcomes. you will identify training and developmental needs and drive suitable training initiatives that build loyalty to the firm.training managers work across many departments to get employees up to speed in both their specific vertical, and the overall needs of the company. training managers often have backgrounds in business, leadership, human resources, development and education. training manager responsibilities include enhancing employees’ skills, performance, productivity and quality of work.responsibilities: identify and assess future and current training needs through job analysis, career paths, annual performance appraisals and consultation with line managers, draw an overall or individualized training and development plan that addresses needs and expectations, deploy a wide variety of training methods, condu

In [115]:
df["job description"][0]

'job brief: we are looking for a competent collection specialist to contact clients and collect outstanding payments. you will struck a balance between maintaining trustful relationships, and ensuring timely payments. our collection specialist should exhibit professionalism and trustworthiness. you should have excellent communication and negotiation skills, as well as an ability to work independently.responsibilities: monitor accounts to identify outstanding debts, investigate historical data for each debt or bill, find and contact clients to ask about their overdue payments, take actions to encourage timely debt payments, process payments and refunds, resolve billing and customer credit issues, update account status records and collection efforts, report on collection activity and accounts receivable status, requirements: <collection-specialist-interview-questions">proven experience as a collection specialist or similar role, knowledge of billing procedures and collection techniques (

In [116]:
df["keyword_with_jobdesc"] = df["job description"].str.findall('|'.join(lower_keyword_list)).apply(set).str.join(', ')
df.head(20)

,title,job description,questions,number_of_questions,keyword_with_jobdesc
0,Collection Specialist,job brief: we are looking for a competent coll...,[['A debtor has missed six months of payments....,16,professional
1,Billing Analyst,job brief: we are looking for a billing analys...,[['How would you create a report on debts from...,14,"accounting, sql"
2,Tax Accountant,job brief: we are looking for a tax accountant...,[['What would you do if you received a B-Notic...,14,"accounting, finance"
3,Accounts Receivable Manager,job brief: we are looking for an accounts rece...,[['What would be the first thing you’d check i...,17,"finance, accounting, forecasting, quality"
4,Cost Accountant,job brief: we are looking for a cost accountan...,"[['If you could use only Excel, how would you ...",15,"accounting, finance"
5,Certified Public Accountant (CPA),job brief: we are looking for an experienced c...,[['If you missed the deadline for paying our c...,15,"accounting, finance"
6,Accounting Supervisor,job brief: we are looking for an accounting su...,[['What measures would you take if you discove...,13,"accounting, finance"
7,Senior Auditor,job brief: we are looking for a senior auditor...,[['What would you do if a client/manager asked...,14,"professional, accounting, finance"
8,Billing Specialist,job brief: we are looking for a competent bill...,[['Describe your typical day at your current/p...,14,"accounting, finance"
9,Billing Clerk,job brief: we are looking for a reliable billi...,[['What data would you present to inform your ...,16,accounting


Once we have the keywords that match the job description, we create a list out of them for further processing:

In [118]:
df["keyword_with_jobdesc"]=df["keyword_with_jobdesc"].str.split(", ")
df.head(20)

,title,job description,questions,number_of_questions,keyword_with_jobdesc
0,Collection Specialist,job brief: we are looking for a competent coll...,[['A debtor has missed six months of payments....,16,[professional]
1,Billing Analyst,job brief: we are looking for a billing analys...,[['How would you create a report on debts from...,14,"[accounting, sql]"
2,Tax Accountant,job brief: we are looking for a tax accountant...,[['What would you do if you received a B-Notic...,14,"[accounting, finance]"
3,Accounts Receivable Manager,job brief: we are looking for an accounts rece...,[['What would be the first thing you’d check i...,17,"[finance, accounting, forecasting, quality]"
4,Cost Accountant,job brief: we are looking for a cost accountan...,"[['If you could use only Excel, how would you ...",15,"[accounting, finance]"
5,Certified Public Accountant (CPA),job brief: we are looking for an experienced c...,[['If you missed the deadline for paying our c...,15,"[accounting, finance]"
6,Accounting Supervisor,job brief: we are looking for an accounting su...,[['What measures would you take if you discove...,13,"[accounting, finance]"
7,Senior Auditor,job brief: we are looking for a senior auditor...,[['What would you do if a client/manager asked...,14,"[professional, accounting, finance]"
8,Billing Specialist,job brief: we are looking for a competent bill...,[['Describe your typical day at your current/p...,14,"[accounting, finance]"
9,Billing Clerk,job brief: we are looking for a reliable billi...,[['What data would you present to inform your ...,16,[accounting]


### Create a counter of keywords per question

In [158]:
column_list=[]
question_key_count={}
for role in df["questions"]:
    new_dict={}
    for question in role:
        new_dict[question]=0
    column_list.append(new_dict)

i=0
keyword_count=0
for role_dict in column_list:  
    for question in role_dict.keys():
        for keyword in df["keyword_with_jobdesc"][i]: 
            if keyword in question:  
                keyword_count+=1
        role_dict[question]=keyword_count
        keyword_count=0
    i+=1
            
print(column_list[1])
print(len(column_list))

{"['How would you create a report on debts from the previous quarter": 0, 'ow would you contact clients who haven’t paid their bills': 0, 'f a long-term client had a debt at the end of the fiscal year, how would you address it with him/her': 0, 'ow would you make sure we collect invoices in a timely manner': 0, 'ow would you handle an invoice discrepancy with a client': 0, 'riefly explain the financial transactions involved in the billing process and your experience with each': 0, 'hat accounting software have you used': 1, 'ow do you keep updated records of accounts receivable from clients': 0, 'ow do you ensure financial information remains confidential': 0, 'hat are your responsibilities during annual audits': 0, 'ave you ever made a mistake at work (e.g. a wrong calculation)? How did you figure it out and what happened': 0, 'ow do you manage to meet tight deadlines': 0, 'ow do you stay up-to-date with new financial regulations? Do you read any specific blogs, magazines or newspaper

In [121]:
df["keywords_per_question"]=column_list
df.head()

,title,job description,questions,number_of_questions,keyword_with_jobdesc,keywords_per_question
0,Collection Specialist,job brief: we are looking for a competent coll...,[['A debtor has missed six months of payments....,16,[professional],{'['A debtor has missed six months of payments...
1,Billing Analyst,job brief: we are looking for a billing analys...,[['How would you create a report on debts from...,14,"[accounting, sql]",{'['How would you create a report on debts fro...
2,Tax Accountant,job brief: we are looking for a tax accountant...,[['What would you do if you received a B-Notic...,14,"[accounting, finance]",{'['What would you do if you received a B-Noti...
3,Accounts Receivable Manager,job brief: we are looking for an accounts rece...,[['What would be the first thing you’d check i...,17,"[finance, accounting, forecasting, quality]",{'['What would be the first thing you’d check ...
4,Cost Accountant,job brief: we are looking for a cost accountan...,"[['If you could use only Excel, how would you ...",15,"[accounting, finance]","{'['If you could use only Excel, how would you..."


In [122]:
df["keywords_per_question"][1]

{"['How would you create a report on debts from the previous quarter": 0,
 'ow would you contact clients who haven’t paid their bills': 0,
 'f a long-term client had a debt at the end of the fiscal year, how would you address it with him/her': 0,
 'ow would you make sure we collect invoices in a timely manner': 0,
 'ow would you handle an invoice discrepancy with a client': 0,
 'riefly explain the financial transactions involved in the billing process and your experience with each': 0,
 'hat accounting software have you used': 1,
 'ow do you keep updated records of accounts receivable from clients': 0,
 'ow do you ensure financial information remains confidential': 0,
 'hat are your responsibilities during annual audits': 0,
 'ave you ever made a mistake at work (e.g. a wrong calculation)? How did you figure it out and what happened': 0,
 'ow do you manage to meet tight deadlines': 0,
 'ow do you stay up-to-date with new financial regulations? Do you read any specific blogs, magazines 

### Create a TFIDF for the questions words in order to get a score of them

In [124]:
df_test=pd.read_csv("Dataset.csv")

In [125]:
tfidf_questions=df_test["questions"]

In [127]:
tf_idf_weighting = TfidfVectorizer()
tf_idf_test = tf_idf_weighting.fit_transform(tfidf_questions)
tf_idf_dt_matrix = pd.DataFrame(tf_idf_test.A, columns=tf_idf_weighting.get_feature_names())
tf_idf_dt_matrix

,00,000,10,100,1096,1099,12,14,14001,15,...,yes,yet,yii,you,young,younger,your,yourself,zoho,zone
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.399921,0.0,0.0,0.050953,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.319325,0.0,0.0,0.048434,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.122246,0.122246,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.369459,0.0,0.0,0.108962,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.419516,0.0,0.0,0.121476,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.086231,0.000000,0.0,0.372815,0.0,0.0,0.018849,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.096141,0.0,0.529787,0.0,0.0,0.000000,0.0,0.0,0.0
279,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.327401,0.0,0.0,0.109772,0.0,0.0,0.0
280,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.249732,0.0,0.0,0.033143,0.0,0.0,0.0
281,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.269223,0.0,0.0,0.153914,0.0,0.0,0.0


In [130]:
document = 2
print("25 most important terms for role", df_test.iloc[document]['title'])
print(tf_idf_dt_matrix.iloc[:, np.argsort(tf_idf_dt_matrix.loc[document])[::-1]].iloc[document][:25])

print("25 least important terms for role", df_test.iloc[document]['title'])
print(tf_idf_dt_matrix.iloc[:, np.argsort(tf_idf_dt_matrix.loc[document])[::-1]].iloc[document][-25:])

25 most important terms for role Tax Accountant
you            0.369459
tax            0.281376
notice         0.206878
regulatory     0.193066
what           0.164204
accounting     0.146057
do             0.124023
create         0.123621
how            0.123588
1099           0.122246
1096           0.122246
withholding    0.122246
actually       0.122246
submission     0.122246
attracted      0.122246
forensic       0.122246
reports        0.120501
versus         0.113924
fields         0.113924
irs            0.113924
aware          0.113924
backup         0.113924
returns        0.113924
your           0.108962
fine           0.108019
Name: 2, dtype: float64
25 least important terms for role Tax Accountant
portfolio       0.0
portability     0.0
populations     0.0
popular         0.0
poorly          0.0
poor            0.0
polymorphism    0.0
polling         0.0
policy          0.0
policies        0.0
police          0.0
points          0.0
pointless       0.0
pointers        0.0

In [142]:
df["keyword_with_jobdesc"][1]

['accounting', 'sql']

In [141]:
word_value_dict={}
test=[]
n=1
for keyword in df["keyword_with_jobdesc"][1]:
    if keyword in tf_idf_dt_matrix.columns:
        test.append(tf_idf_dt_matrix[keyword][n])
        #test.append(keyword)
print(test)

[0.08115248515402664, 0.0]


In [137]:
df["keywords_per_question"][1]

{"['How would you create a report on debts from the previous quarter": 0,
 'ow would you contact clients who haven’t paid their bills': 0,
 'f a long-term client had a debt at the end of the fiscal year, how would you address it with him/her': 0,
 'ow would you make sure we collect invoices in a timely manner': 0,
 'ow would you handle an invoice discrepancy with a client': 0,
 'riefly explain the financial transactions involved in the billing process and your experience with each': 0,
 'hat accounting software have you used': 1,
 'ow do you keep updated records of accounts receivable from clients': 0,
 'ow do you ensure financial information remains confidential': 0,
 'hat are your responsibilities during annual audits': 0,
 'ave you ever made a mistake at work (e.g. a wrong calculation)? How did you figure it out and what happened': 0,
 'ow do you manage to meet tight deadlines': 0,
 'ow do you stay up-to-date with new financial regulations? Do you read any specific blogs, magazines 

### Create a list for every rol, containing a dictionary of words and weights for every questions

In [145]:
word_score_list=[]
i=0
for title in df["questions"]:
    role_list=[]
    for question in title:
        new_dict={}
        role_list.append(new_dict)
        for keyword in df["keyword_with_jobdesc"][i]:
            if keyword in tf_idf_dt_matrix.columns and (keyword in question):
                new_dict[keyword]=0
    word_score_list.append(role_list)
    i+=1
        
i=0
for role_list in word_score_list:  
    for quest_dict in role_list:
        for word in quest_dict.keys():
            quest_dict[word]=tf_idf_dt_matrix[word][i]
    i+=1  


print(word_score_list[1])
print(len(word_score_list)) 

[{}, {}, {}, {}, {}, {}, {'accounting': 0.08115248515402664}, {}, {}, {}, {}, {}, {}, {}]
283


### Create a list for each rol, containing the values of the sum of the keywords weights for each question

In [161]:
list_of_points=[]
for role in word_score_list:
    role_list=[]
    for word_dict in role:
        sum=0
        for word in word_dict.keys():
            sum+=word_dict[word]
        role_list.append(sum)
    
    list_of_points.append(role_list)


print(list_of_points[1])
len(list_of_points)


[0, 0, 0, 0, 0, 0, 0.08115248515402664, 0, 0, 0, 0, 0, 0, 0]


283

### Create a dictionary for each rol, with the questions and the scores of each one of them

In [151]:
question_score_list=[]
t=0
for title in df["questions"]:
    quest_dict={}
    q=0
    for question in title:
        score=list_of_points[t][q]
        quest_dict[question]=score
        q+=1
    t+=1

    question_score_list.append(quest_dict)


print(question_score_list[1])
print(len(question_score_list))

{"['How would you create a report on debts from the previous quarter": 0, 'ow would you contact clients who haven’t paid their bills': 0, 'f a long-term client had a debt at the end of the fiscal year, how would you address it with him/her': 0, 'ow would you make sure we collect invoices in a timely manner': 0, 'ow would you handle an invoice discrepancy with a client': 0, 'riefly explain the financial transactions involved in the billing process and your experience with each': 0, 'hat accounting software have you used': 0.08115248515402664, 'ow do you keep updated records of accounts receivable from clients': 0, 'ow do you ensure financial information remains confidential': 0, 'hat are your responsibilities during annual audits': 0, 'ave you ever made a mistake at work (e.g. a wrong calculation)? How did you figure it out and what happened': 0, 'ow do you manage to meet tight deadlines': 0, 'ow do you stay up-to-date with new financial regulations? Do you read any specific blogs, maga

In [162]:
df["scored_questions"]=question_score_list
df.head()

,title,job description,questions,number_of_questions,keyword_with_jobdesc,keywords_per_question,scored_questions
0,Collection Specialist,job brief: we are looking for a competent coll...,[['A debtor has missed six months of payments....,16,[professional],{'['A debtor has missed six months of payments...,{'['A debtor has missed six months of payments...
1,Billing Analyst,job brief: we are looking for a billing analys...,[['How would you create a report on debts from...,14,"[accounting, sql]",{'['How would you create a report on debts fro...,{'['How would you create a report on debts fro...
2,Tax Accountant,job brief: we are looking for a tax accountant...,[['What would you do if you received a B-Notic...,14,"[accounting, finance]",{'['What would you do if you received a B-Noti...,{'['What would you do if you received a B-Noti...
3,Accounts Receivable Manager,job brief: we are looking for an accounts rece...,[['What would be the first thing you’d check i...,17,"[finance, accounting, forecasting, quality]",{'['What would be the first thing you’d check ...,{'['What would be the first thing you’d check ...
4,Cost Accountant,job brief: we are looking for a cost accountan...,"[['If you could use only Excel, how would you ...",15,"[accounting, finance]","{'['If you could use only Excel, how would you...","{'['If you could use only Excel, how would you..."


In [163]:
 first_question=df["scored_questions"][1]

In [164]:
{k: v for k, v in sorted( first_question.items(), key=lambda item: item[1],reverse=True)}

{'hat accounting software have you used': 0.08115248515402664,
 "['How would you create a report on debts from the previous quarter": 0,
 'ow would you contact clients who haven’t paid their bills': 0,
 'f a long-term client had a debt at the end of the fiscal year, how would you address it with him/her': 0,
 'ow would you make sure we collect invoices in a timely manner': 0,
 'ow would you handle an invoice discrepancy with a client': 0,
 'riefly explain the financial transactions involved in the billing process and your experience with each': 0,
 'ow do you keep updated records of accounts receivable from clients': 0,
 'ow do you ensure financial information remains confidential': 0,
 'hat are your responsibilities during annual audits': 0,
 'ave you ever made a mistake at work (e.g. a wrong calculation)? How did you figure it out and what happened': 0,
 'ow do you manage to meet tight deadlines': 0,
 'ow do you stay up-to-date with new financial regulations? Do you read any specific

# Enrique approach

In [294]:
# Get the questions in the dictionary format needed
preguntas = (dict(enumerate(all_questions)))
preguntas

{0: "['A debtor has missed six months of payments. When you call the individual, they bring up financial difficulties and get emotional. How do you handle this?",
 1: 'What payback plan would you recommend for a long-term client whose payments are due to be collected in X months?',
 2: 'If you called a client to update them on their payment status and they were aggressive or rude to you, how would you handle it?',
 3: 'Can you think of a few common excuses that debtors make? How would you respond to each of them?',
 4: 'What collection methods do you know? Which technique do you think would be more effective for our company/clients?',
 5: 'How do you keep historical data for each account?',
 6: 'How often do you follow up with delinquent account holders and what’s your approach when you get in touch?',
 7: 'Mention three things a Collection Specialist should avoid saying when speaking to a debtor.',
 8: 'How do you keep track of all payments?',
 9: 'What software do you use to increase

In [295]:
tfidf = TfidfVectorizer(vocabulary = Keywords, stop_words = 'english', ngram_range=(1,2))
#tfs = tfidf.fit_transform([docA, docB])
tfs = tfidf.fit_transform(preguntas.values())

In [296]:
feature_names = tfidf.get_feature_names_out()
preguntas_index = [n for n in preguntas]
rows, cols = tfs.nonzero()
for row, col in zip(rows, cols):
    print((feature_names[col], preguntas_index[row]), tfs[row, col])

('payments', 0) 1.0
('collected', 1) 0.7601190944419842
('payments', 1) 0.6497837811647024
('payments', 8) 1.0
('persuaded', 12) 1.0
('timely', 19) 0.7479162774082595
('invoices', 19) 0.663793071662978
('involved', 21) 0.5558906663067212
('transactions', 21) 0.6116679422165304
('billing', 21) 0.5628924369519144
('accounting', 22) 1.0
('updated', 23) 0.6840452018161833
('accounts receivable', 23) 0.7294396218140725
('regulations', 28) 1.0
('motivated', 29) 1.0
('forecasting', 31) 0.7193989891642376
('tax', 31) 0.69459707340981
('tax', 32) 1.0
('regulatory', 33) 1.0
('accounting', 34) 1.0
('accounting', 39) 0.4750769515583856
('auditing', 39) 0.6726049774054396
('tax', 39) 0.5673662260545823
('regulatory', 40) 1.0
('saved', 42) 1.0
('billing', 44) 1.0
('accounts receivable', 46) 1.0
('hired', 47) 0.677799862190945
('accounts receivable', 47) 0.7352464531121083
('payments', 48) 1.0
('set', 49) 0.7273742258659908
('goals', 49) 0.6862410185538682
('accounting', 50) 1.0
('reconciliation', 52

In [297]:
import pandas as pd
df = pd.DataFrame(tfs.todense(), columns=feature_names, index=preguntas_index)
df

,teaching,pharmaceutical,erp,fulfillment,positioning,tax,service delivery,investigate,editorial,account management,...,tireless,unique,unprecedented,vigorous,visionary,vivid,well-grounded,wholehearted,world-class,zealous
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [298]:
df['total']= df.sum(axis=1)

In [299]:
df['total'].nlargest(n=10)

2333    2.222944
3424    2.209646
429     2.201810
1406    2.191801
2802    1.997870
3345    1.995789
3452    1.995789
1927    1.990829
3106    1.984608
3140    1.984608
Name: total, dtype: float64

In [300]:
Keywords_from_cv_parser = skills

In [301]:
df_skills = df.filter(Keywords_from_cv_parser)

In [302]:
df_skills.head()

,finance,analysis,presentations,logical,performance metrics,skilled,enhanced,documentation,quality,metrics,managed,data analysis,marketing,quality control
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [303]:
df_skills['sum_of_coefficients_per_questions']= df_skills.sum(axis=1)

In [304]:
df_skills.head()

,finance,analysis,presentations,logical,performance metrics,skilled,enhanced,documentation,quality,metrics,managed,data analysis,marketing,quality control,sum_of_coefficients_per_questions
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [305]:
keys = df_skills['sum_of_coefficients_per_questions'].nlargest(n=20).index.tolist()

In [306]:
Best_Questions = list(map(preguntas.get, keys) )

In [307]:
Best_Questions

['What metrics do you monitor on a daily basis? How do you present them to the Marketing department?',
 'When you need to explain marketing results, do you prefer reports or presentations? Why?',
 'How would you measure performance and what performance metrics have you used before?',
 'How would you rate your data analysis skills, on a scale of 1 to 10?',
 'How do you define quality in a plant? What quality control processes have you implemented in the past?',
 'When presenting performance metrics, how do you make sure your audience understands technical terms?',
 'What sales and marketing metrics do you track? Why?',
 'Tell us about a time you discovered quality control issues in an operation. How did you address these issues?',
 'How would you organize data into spreadsheets to track performance metrics for each department? What kinds of reports would you produce?',
 'What marketing metrics do you regularly track? How do you organize and parse the data? (e.g. in spreadsheets)',
 "['W